(introduction-to-simphony)=
# Introduction to simphony

Simphony is a Python package for defining and simulating photonic circuits, so having a basic understanding of Python will be helpful. In order to get started you will need to setup a python environment with simphony installed. If you are new to Python, we recommend using [Anaconda](https://www.anaconda.com/products/individual) to manage your Python environment. Once you have Anaconda installed, you can create a new environment and install the [``simphony``](https://pypi.org/project/simphony/) package by running the following commands in your terminal:

```bash
conda create -n simphony python=3.11
conda activate simphony
pip install simphony
```

Our goal with this tutorial is to give some of the background of the basics of simphony in order to simulate a very simple photonic circuit. We'll go through the typical objects found in every circuit definition. We'll also show how to simulate the circuit and obtain the results.

```{note}
Simphony uses SPICE-like concepts--such as components, ports, and nets--to define circuits. This should make simphony
intuitive for all those familiar with SPICE, which is commonly used to define electronic circuits. 
```

## Models

Models are the basic components in simphony, which are used to represent an element in a photonic circuit. The base Model class provides a common interface for all models, including methods to get and set the scattering parameters, get references to ports, rename ports, and check if a model is connected to any others. It also 
provides methods to convert the model to a scikit-rf network, which can be useful for further analysis and 
simulation.

```{eval-rst}
More detailed information about the Model parent class can be found on its doc page :py:class:`simphony.models.Model`:
```

All models in Simphony extend this parent class, but redefine ports, and s-parameters to match the device they represent. Simphony has a number of built-in models, but you can also create your own custom models by extending this Model class. Here we make a simple custom model with two ports and one parameter:

In [5]:
import numpy as np
from simphony.models import Model

class CustomModel(Model):
    onames = ["o0", "o1"] # indicates the names of the optical ports of the model

    def __init__(self, param: float = 0.5): # this model will have one parameter
        self.param = param

    def s_params(self, wl):
        # a simple wavelength independent s-matrix
        smatrix = np.array([[0, -1j * np.sqrt(self.param)],[np.sqrt(self.param), 0]])
        return np.stack([smatrix] * len(wl), axis=0) # repeats smatrix for each wavelength
    
mymodel = CustomModel(param=0.25)
mymodel.s_params(wl=np.linspace(1.5, 1.6, 3))

array([[[0. +0.j , 0. -0.5j],
        [0.5+0.j , 0. +0.j ]],

       [[0. +0.j , 0. -0.5j],
        [0.5+0.j , 0. +0.j ]],

       [[0. +0.j , 0. -0.5j],
        [0.5+0.j , 0. +0.j ]]])

```{note}
All s_params in simphony are an array of shape (n_wl, n_ports, n_ports). The first dimension being the number of wavelength points and the next two being the scattering parameters for each port. For example S[2, 0, 1] is the scattering parameter from port 0 to port 1 at wavelength point corresponding to index 2.
```{note}

A basic model has no ``__init__()`` function. It is only required if the model takes in parameters (width or length, for example) that  affect the scattering parameters.
```

## Using Pre-built Models

```{eval-rst}
Simphony includes some pre-built models that can be used to build photonic circuits :py:mod:`simphony.libraries`:
``` 

One library we use comes from the [SiEPIC PDK](https://github.com/SiEPIC) (developed at the University of British Columbia). This library contains the s-parameters for a number of photonic components that were fabricated and measured for a number of different parameters.

For example lets instantiate two different models for a waveguide of different lengths:

In [8]:
from simphony.libraries import siepic

# waveguide of 2.5 mm length
wg1 = siepic.Waveguide(length=100, height=220)
# waveguide of 7.5 mm length
wg2 = siepic.Waveguide(length=100, height=210)

```{eval-rst}
These are both :py:class:`~simphony.libraries.siepic.Waveguide` models. The SiEPIC components are parameterizable, so we can pass different parameters when instantiating them. In this case ``wg1`` will be a thicker waveguide than ``wg2``. The two models will have different s-parameters since their heights are different.
```

```{note}
The convention in simphony is to use microns for units of length.
```

## Creating a Circuit

```{eval-rst}
The :py:class:`~simphony.models.Port` class is used as an interface to connect two models in a circuit. The :py:class:`~simphony.circuit.Circuit` class contains methods that handles connecting ports for you. Let's give an example.
```

The simplest way to connect ports is as follows:

In [9]:
from simphony.circuit import Circuit

wg1 = siepic.Waveguide(length=2500, loss=3)
wg2 = siepic.Waveguide(length=7500, loss=3)

ckt = Circuit()
ckt.connect(wg1, wg2)

This will connect the first unconnected optical port on each component together. However, if we want the first port of ``wg1`` to
be an input, and instead connect its second port to ``wg2`` as an output, we have to connect the ports explicitly (recalling that ports are 0-indexed):

In [10]:
wg1 = siepic.Waveguide(length=2500, loss=3)
wg2 = siepic.Waveguide(length=7500, loss=3)

ckt = Circuit()
ckt.connect(wg1.o(1), wg2.o(0))

By default, a model instantiates its ports with names "o0", "o1", etc. Here we specify "o1" of ``wg1`` must
connect to "o0" of ``wg2``. 

In [11]:
wg1 = siepic.Waveguide(length=2500, loss=3)
wg2 = siepic.Waveguide(length=7500, loss=3)

ckt = Circuit()
ckt.connect(wg1.o("o1"), wg2.o("o0"))

We can also rename ports for semantic clarity. Also, anytime we specify a model without specifying a port, the first unconnected port is used.

In [12]:
wg1 = siepic.Waveguide(length=2500, loss=3)
wg2 = siepic.Waveguide(length=7500, loss=3)

# Pass in a list of strings to rename the ports of component1 in the same order 
# as they're defined in the model
wg1.rename_oports(["input", "output"])

ckt = Circuit()
# Use the next unconnected port of component2
ckt.connect(wg1.o("output"), wg2)

Here, we do the same as the previous example, except that we rename the two ports of ``component1`` to "input" and
"output", and then connect "output" to ``component2``. We do not need to explicitly specify "o0" for ``component2``,
since that is the first unconnected port.

With this connection, we now have a rudimentary circuit to run simulations on.

## Simulation

```{eval-rst}
:py:mod:`simphony.simulators` provides a collection of simulators that connect to an input and output pin on a
circuit, then perform a subnetwork growth algorithm (a series of matrix operations). The results show us what output light comes out of the circuit for given inputs of light.
```

<!-- The simulation process modifies pins and components, so
simulators actually copy the circuit they are passed in order to preserve the original circuit. -->

Let's run a simple sweep simulation on the circuit we have created:

In [13]:
from simphony.simulation import ClassicalSim

# Create a simulation and add a laser and detector
sim = ClassicalSim(ckt, wl=1.55)
laser = sim.add_laser(ports=ckt.o("input"), power=1.0)
detector = sim.add_detector(ports=ckt.o(1))

# Run the simulation
result = sim.run()

# Since the total wg length is 1 cm and the loss is 3 dB/cm, the power should be 50%.
print(f"Power transmission: {abs(result.output[0, 0])**2}")

Power transmission: 0.5011872336272725


/home/chuuni/simphony/simphony/simulation/simdevices.py:67: UserWarning: Responsivity is not yet implemented, so it is ignored.
  warnings.warn("Responsivity is not yet implemented, so it is ignored.")


We instantiated up our simulator with our circuit, adding a laser input to the "input" port of ``wg1`` and placing a detector on
``wg2``. Our sweep simulation passed input light on a range of wavelengths from 1.5 microns to
1.6 microns, and now ``result`` contains what frequencies came out of our circuit. We can use these results however we like.

In order to view the results, we can use the ``matplotlib`` package to graph our output, but that will be demonstrated
in following tutorials. For this tutorial, we're done!